In [ ]:
#Download the dataset
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

In [ ]:
!unzip /content/drive/MyDrive/ibm/Flowers-Dataset

Archive:  /content/drive/MyDrive/ibm/Flowers-Dataset.zip
replace flowers/daisy/100080576_f52e8ee070_n.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#image_augmentation

In [ ]:
train_ds=ImageDataGenerator(rescale=1./255,
                            zoom_range=0.2,
                            horizontal_flip=True)

In [ ]:
test_ds=ImageDataGenerator(rescale=1./255)

In [ ]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio("flowers",output="train_dss",seed=1337,ratio=(0.8,0.2),group_prefix=None,move=False)

Copying files: 4317 files [00:01, 3453.51 files/s]


In [ ]:
x_train=train_ds.flow_from_directory(r'train_dss/train',
                                     target_size=(64,64),
                                     class_mode='categorical',
                                     batch_size=100)

Found 3452 images belonging to 5 classes.


In [ ]:
x_val=test_ds.flow_from_directory(r'train_dss/val',
                                  target_size=(64,64),
                                  class_mode='categorical',
                                  batch_size=100)

Found 865 images belonging to 5 classes.


In [ ]:
x_train.class_indices

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}

In [ ]:
#create model

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [ ]:
model=Sequential()

In [ ]:
#Add layers

In [ ]:
model.add(Convolution2D(32,(3,3),kernel_initializer='random_uniform',activation='relu',strides=(1,1),input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(300,activation='relu'))
model.add(Dense(300,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 300)               9225900   
                                                                 
 dense_1 (Dense)             (None, 300)               90300     
                                                                 
 dense_2 (Dense)             (None, 5)                 1505      
                                                        

In [ ]:
#compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#fit the model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

In [ ]:
early_stopping=EarlyStopping(monitor='val_accuracy',
                             patience=5)
reduce_lr=ReduceLROnPlateau(monitor='val_accuracy',
                            patience=5,
                            factor=0.5,
                            min_lr=0.00001)
callback=[reduce_lr,early_stopping]

In [ ]:
model.fit_generator(x_train,
                    steps_per_epoch=len(x_train),
                    epochs=15,
                    validation_data=x_val,
                    validation_steps=len(x_val))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/15
35/35 [==============================] - 26s 714ms/step - loss: 1.3533 - accuracy: 0.4050 - val_loss: 1.2919 - val_accuracy: 0.5029
Epoch 2/15
35/35 [==============================] - 24s 691ms/step - loss: 1.1163 - accuracy: 0.5403 - val_loss: 1.1711 - val_accuracy: 0.5422
Epoch 3/15
35/35 [==============================] - 24s 696ms/step - loss: 1.0346 - accuracy: 0.5973 - val_loss: 1.1061 - val_accuracy: 0.5526
Epoch 4/15
35/35 [==============================] - 25s 695ms/step - loss: 0.9726 - accuracy: 0.6144 - val_loss: 0.9866 - val_accuracy: 0.6150
Epoch 5/15
35/35 [==============================] - 24s 696ms/step - loss: 0.8985 - accuracy: 0.6509 - val_loss: 0.9801 - val_accuracy: 0.6382
Epoch 6/15
35/35 [==============================] - 24s 692ms/step - loss: 0.8412 - accuracy: 0.6770 - val_loss: 1.0169 - val_accuracy: 0.6382
Epoch 7/15
35/35 [==============================] - 26s 728ms/step - loss: 0.8051 - accuracy: 0.6886 - val_loss: 0.9958 - val_accuracy: 0.6254

In [ ]:
#save the model
model.save("Flowers.h5")

In [ ]:
#test the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
img1=image.load_img(r'flowers/rose/110472418_87b6a3aa98_m.jpg')

In [ ]:
img1=img1.resize((64,64))

In [ ]:
x=image.img_to_array(img1)

In [ ]:
x

array([[[154.,   1.,   2.],
        [167.,   1.,   2.],
        [160.,   0.,   2.],
        ...,
        [ 71.,   1.,   3.],
        [ 18.,   1.,   2.],
        [ 11.,   0.,   0.]],

       [[141.,   2.,   1.],
        [127.,   1.,   2.],
        [164.,   1.,   2.],
        ...,
        [  7.,   1.,   2.],
        [  9.,   1.,   1.],
        [  9.,   0.,   1.]],

       [[149.,   1.,   1.],
        [119.,   1.,   2.],
        [150.,   1.,   3.],
        ...,
        [  9.,   1.,   1.],
        [  9.,   1.,   0.],
        [ 13.,   0.,   1.]],

       ...,

       [[173.,   2.,   1.],
        [178.,   1.,   0.],
        [168.,   2.,   1.],
        ...,
        [170.,   2.,   4.],
        [199.,   1.,   3.],
        [228.,   0.,   2.]],

       [[154.,   2.,   1.],
        [146.,   2.,   1.],
        [131.,   2.,   1.],
        ...,
        [196.,   1.,   2.],
        [206.,   1.,   1.],
        [221.,   2.,   1.]],

       [[159.,   1.,   1.],
        [141.,   1.,   2.],
        [135.,  

In [ ]:
import numpy as np

In [ ]:
x=np.expand_dims(x,axis=0)

In [ ]:
y=np.argmax(model.predict(x))

1/1 [==============================] - 0s 28ms/step


In [ ]:
y

2

In [ ]:
index=['daisy','dandelion','rose','sunflower','tulip']
index[y]

'rose'